# TensorFlow 

### 图

### 层 Dense

In [6]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
class Dense(tf.Module):
    def __init__(self, in_feature, out_feature, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_feature,out_feature]),name='w')
        self.b = tf.Variable(tf.zeros([out_feature]), name='b')
    def __call__(self, x):
        y = tf.reduce_sum(self.w) + self.b
        return tf.nn.sigmoid(y)

In [8]:
dense1 = Dense(in_feature=3, out_feature=1,name = 'dense1')
dense1(tf.constant([[0.0288],[-0.3256],[0.5925]]))

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.49001592], dtype=float32)>

In [11]:
class MySequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_feature=3, out_feature=3)
    self.dense_2 = Dense(in_feature=3, out_feature=2)
        
  @tf.function()
  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model!
my_model = MySequentialModule(name="the_model")

# Call it, with random results
print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))

Model results: tf.Tensor([0.19743785 0.19743785], shape=(2,), dtype=float32)


In [12]:
# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = "logs/func/%s" % stamp
writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace
# Otherwise the summary will not see the graph.
new_model = MySequentialModule()

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
with writer.as_default():
  tf.summary.trace_export(
      name="my_func_trace",
      step=0,
      profiler_outdir=logdir)

tf.Tensor([0.39643517 0.39643517], shape=(2,), dtype=float32)


In [13]:
%tensorboard --logdir logs/func

Reusing TensorBoard on port 6006 (pid 14132), started 2:53:39 ago. (Use '!kill 14132' to kill it.)

### 存储和恢复